Use Python to fetch country debt profiles. Use a horizontal bar chart, the y-axis is the list of all countries in the world (for example, Japan, Korea, Philippines, China, and more), and the x-axis is the total debt in $.  Make a ranking label on the outside and the right part of the bar, followed by the number of debt$ label

In [ ]:
import tradingeconomics as te
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# 1) TradingEconomics API login
# -------------------------
te.login('YOUR_API_KEY:YOUR_API_SECRET')  # replace with your key

# -------------------------
# 2) Fetch government debt data (latest) using correct parameter
# -------------------------
df = te.getIndicatorData(indicators='GovernmentDebt', output_type='df')

# -------------------------
# 3) Keep the latest observation per country
# -------------------------
df_latest = df.sort_values('Date').groupby('Country').tail(1)
df_latest = df_latest.dropna(subset=['Value'])
df_latest = df_latest.rename(columns={'Value': 'DebtUSD'})

# -------------------------
# 4) Sort ascending by debt and add ranking
# -------------------------
df_plot = df_latest.sort_values(by='DebtUSD', ascending=True)
df_plot['Rank'] = df_plot['DebtUSD'].rank(method='first', ascending=False).astype(int)

# -------------------------
# 5) Plot horizontal bar chart
# -------------------------
plt.figure(figsize=(14, max(8, len(df_plot)/3)))
bars = plt.barh(df_plot['Country'], df_plot['DebtUSD'], color='steelblue')

plt.title('National Government Debt by Country (Latest)')
plt.xlabel('Government Debt (USD)')

for bar, rank, debt in zip(bars, df_plot['Rank'], df_plot['DebtUSD']):
    width = bar.get_width()
    if debt >= 1e12:
        label = f"{debt/1e12:,.2f}T"
    elif debt >= 1e9:
        label = f"{debt/1e9:,.2f}B"
    else:
        label = f"{debt:,.0f}"
    plt.text(
        width * 1.01,
        bar.get_y() + bar.get_height()/2,
        f"#{rank} {label}",
        va='center',
        fontsize=8
    )

plt.tight_layout()
plt.show()
